# Tutorial: Simple Fishing Game

To get you started, I defined a super-simple fishing game, based on the first three locations
in __Action Castle__, which is a text adventure game in a fun book by Jared A. Sorensen called
[Parsley](http://www.memento-mori.com/parsely-products/parsely-pdf).


<img src="tutorial-map.png" width="500"/>



The goal of this game is simple: help your character find some food to eat because they are hungry!

## The Text Adventures Library

We will be using a framework designed for this class called _Text Adventures_, which is included in this repo.

### High-Level

The main concepts we are concerned with when building a game with Text Adventures are:

* Things: _locations, items, and characters_
* Actions: things a character can do
* Blocks: things that prevent a character from going somewhere
* Commands: how a user interacts with the game

### Things

_Things_ can have _properties_. A property is some detail about the thing that influences how other things can interact with it. A poisonous potion might have the property `is_poisonous` set, or a locked door might have `is_locked` set. Properties are designed to be flexible and easily extensible without significant code changes.

_Things_ can also have commands associated with them. A rosebush might have a command that lets characters pick a rose from the bush, and that rose might have a command for smelling it.

All _things_ are stored in `text_adventures.things`.

### Actions

An action is something a user does to change the game's state, such as changing location, eating food, or attacking something.

When a user enters a command, the game will parse the command to determine its intent and route it to the appropriate action. This means a user can type _go west_ and the game will work out that this means `text_adventures.actions.Go` and it will determine if the character can move west based on where the character is.

The game's state might change each time an action is executed, so action instances are created for each command and the game's state is re-evaluated to determine if the action will be successful or fail.

### Blocks

Blocks are useful for implementing things like a locked door or if some direction is guarded by a monster.

If a user attempts to enter a location that is blocked from where they are, the command will fail with some information about why. It's up to the game designer as to whether or not a block can be removed.

### Install required libraries

In [10]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 8.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


### Import Text Adventures

In [1]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

## Locations

A location is a place that characters can go to. Locations can be connected to other locations, creating a graph of places to visit, eg. the game map.

The 3 locations from **Action Castle** are:

* a cottage, your home
* a garden path
* a pond where you can fish to the south

In [2]:
cottage = things.Location("Cottage", "You are standing in a small cottage.")

garden_path = things.Location(
    "Garden Path", "You are standing on a lush garden path. There is a cottage here.")

fishing_pond = things.Location(
    "Fishing Pond", "You are at the edge of a small fishing pond.")

courtyard = things.Location(
    "Courtyard", "You are standing in the castle courtyard. A guard blocks the way east.")

great_feasting_hall = things.Location(
    "Great Feasting Hall", "You are in the Great Feasting Hall, long tables filled with food line the room.")

drawbridge = things.Location(
    "Drawbridge", "You are standing on the drawbridge.")

tower_stairs = things.Location(
    "Tower Stairs", "You are at the bottom of a spiraling tower staircase.")

dungeon_stairs = things.Location(
    "Dungeon Stairs", "You stand at the top of a dark stairway leading down into the dungeon.")

dungeon = things.Location(
    "Dungeon", "You are in a dark dungeon. Ancient runes are carved into the walls.")

tower_top = things.Location(
    "Tower Top", "You are at the top of the tower. A beautiful princess is here.")

throne_room = things.Location(
    "Throne Room", "You are in a magnificent throne room. A golden throne sits at the far end.")

We'll also add a 4th location, a cliff. If the player visits the cliff, they will fall off and that will end the game, so be careful!

**Note**: the `game_over` property is set on the cliff.

In [3]:
cliff = things.Location(
    "Cliff",
    "There is a steep cliff here. You fall off the cliff and lose the game. THE END",
)
cliff.set_property("game_over", True)  # RIP 🪦☠️


### Connecting Locations

From the cottage you can go _out_ to the garden path, and from there you can go _west_ to a cliff or _south_ to a fishing pond.


In [4]:
cottage.add_connection("out", garden_path)
garden_path.add_connection("south", fishing_pond)
garden_path.add_connection("north", courtyard)
courtyard.add_connection("west", drawbridge)
courtyard.add_connection("up", tower_stairs)
courtyard.add_connection("down", dungeon_stairs)
courtyard.add_connection("east", great_feasting_hall)
tower_stairs.add_connection("up", tower_top)
dungeon_stairs.add_connection("down", dungeon)
great_feasting_hall.add_connection("north", throne_room)

**NOTE**: Being able to go _out_ implies that you can also go _in_ from the other direction. The Text Adventures framework will account for that automatically when you use one of the following pairs:

* north/south
* east/west
* up/down
* in/out
* inside/outside

<h2>Blocks</h2>

In [5]:
east_block = blocks.Block("east", "The guard blocks your way east!")
courtyard.add_block("east", east_block)

darkness_block = blocks.Block(
    "down", "It's too dark to go down the dungeon stairs. You might fall!")
dungeon_stairs.add_block("down", darkness_block)

door_block = blocks.Block("up", "The tower door is locked. You need a key.")
tower_stairs.add_block("up", door_block)

## Items

Items can be placed at locations or they can be placed in a character's inventory. Any details about the item that may be interesting should be added as properties.

### Gettable Items

A _gettable_ item is an item that a character can put in their inventory using the _get_ command.

In [6]:
fishing_pole = things.Item("pole", "a fishing pole", "A SIMPLE FISHING POLE.")
cottage.add_item(fishing_pole)

potion = things.Item("potion", "a poisonous potion",
                     "IT'S BRIGHT GREEN AND STEAMING.")
potion.set_property("is_drink", True)
potion.set_property("is_poisonous", True)
potion.add_command_hint("drink potion")
cottage.add_item(potion)

key = things.Item("key", "a brass key",
                  "A brass key that probably opens something important.")
key.set_property("gettable", True)
key.set_property("is_key", True)

branch = things.Item("branch", "a sturdy tree branch",
                     "A solid tree branch, perfect for whacking something.")
branch.set_property("gettable", True)
branch.set_property("is_weapon", True)
garden_path.add_item(branch)

rosebush = things.Item("rosebush", "a rosebush",
                       "THE ROSEBUSH CONTAINS A SINGLE RED ROSE. IT IS BEAUTIFUL.")
rosebush.set_property("gettable", False)
rosebush.set_property("has_rose", True)
rosebush.add_command_hint("pick rose")
garden_path.add_item(rosebush)

rose = things.Item("rose", "a red rose",
                   "A beautiful red rose, perfect for a romantic gesture.")
rose.set_property("gettable", True)
rose.set_property("is_gift", True)

lamp = things.Item("lamp", "a glowing lamp",
                   "A lamp that emits a bright light.")
lamp.set_property("gettable", True)
lamp.set_property("is_light_source", True)
cottage.add_item(lamp)

crown = things.Item("crown", "a golden crown",
                    "A majestic crown fit for royalty.")
crown.set_property("gettable", True)
crown.set_property("is_crown", True)
great_feasting_hall.add_item(crown)

throne = things.Item("throne", "a golden throne",
                     "A magnificent throne, the seat of the kingdom.")
throne.set_property("gettable", False)
throne_room.add_item(throne)

pond = things.Item("pond", "a small fishing pond",
                   "THERE ARE FISH IN THE POND.")
pond.set_property("gettable", False)
pond.set_property("has_fish", True)
pond.add_command_hint("catch fish")
pond.add_command_hint("catch fish with pole")
fishing_pond.add_item(pond)

### Non-gettable Items

Non-gettable items are things that exist at locations but can't be picked up, like a pond. Notice that the `gettable` property is set to False.

In [7]:
# # Put an actual pond at the fishing pond location
# pond = things.Item(
#     "pond",
#     "a small fishing pond",
#     "THERE ARE FISH IN THE POND.",
# )
# pond.set_property("gettable", False)
# pond.set_property("has_fish", True)
# pond.add_command_hint("catch fish")
# pond.add_command_hint("catch fish with pole")
# fishing_pond.add_item(pond)


# # Put a rosebush in the garden
# rosebush = things.Item(
#     "rosebush",
#     "a rosebush",
#     "THE ROSEBUSH CONTAINS A SINGLE RED ROSE.  IT IS BEAUTIFUL.",
# )
# rosebush.set_property("gettable", False)
# rosebush.set_property("has_rose", True)
# rosebush.add_command_hint("pick rose")
# garden_path.add_item(rosebush)


## Characters

A character is some living thing that can perform actions, such as your player, or maybe a mean troll, or perhaps a princess. They can have properties applied to them too, which we'll see when the game sets `is_hungry` on our player below.

In later homeworks we will develop characters more fully so that you can interact with them and they have their own goals and pursue their own interests in the game.

In [8]:
player = things.Character(
    name="The player",
    description="You are a simple peasant destined for greatness.",
    persona="I am on an adventure."
)

guard = things.Character(
    name="Guard",
    description="A stern guard in chainmail blocks the way east. He wears no helmet. A key hangs from his belt.",
    persona="I am guarding the eastern exit."
)
guard.set_property("is_conscious", True)
guard.set_property("has_key", True)
guard.add_to_inventory(key)

courtyard.add_character(guard)

princess = things.Character(
    name="Princess",
    description="A beautiful princess awaits rescue.",
    persona="I am waiting for my true love."
)
tower_top.add_character(princess)

ghost = things.Character(
    name="Ghost",
    description="A translucent ghost blocks your path. Ancient runes on the walls might help.",
    persona="I am guarding this place."
)
dungeon.add_character(ghost)

## The Game

The game class, defined in `text_adventures.games`:

- list of characters, one designated as player
- all locations in the game
- special function to determine if game has been won
- functions for saving and loading game states, which we'll see later.
- a game loop that runs the game, interacting with the user via commands

To create our fishing game, we will subclass `Game` and define the `is_won` function to check when the game is won. In this case, the player starts out hungry, shown by calling `set_property` on the player.

In [9]:

class FishingGame(games.Game):
    def __init__(
        self, start_at: things.Location, player: things.Character, characters=None,
        custom_actions=None
    ):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)
        self.player.set_property("is_hungry", True)

    def is_won(self) -> bool:
        """ 
        Checks whether the player has won the game.  
        For the fishing mini game, this means that the player is no longer hungry.
        """
        if not self.player.get_property("is_hungry"):
            self.parser.ok("You are no longer hungry. You win!")
            return True
        else:
            return False

## Action

You can define your own custom actions for the game. Actions consist of two of three parts:

* the intitialization, which parses the arguments
* the preconditions, which determine if the action can be applied
* the effect, which change the state of the game when the action is successfully applied

In [10]:
class HitGuard(actions.Action):
    ACTION_NAME = "attack"
    ACTION_DESCRIPTION = "Attack the guard with the branch"
    ACTION_ALIASES = ["hit", "strike"]

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command.lower()
        self.character = self.parser.get_character(command)
        self.target = None

        if "guard" not in command.lower():
            raise ValueError("Not attacking guard")

        for char_name, char in self.character.location.characters.items():
            if "guard" in char_name.lower():
                self.target = char
                break

        self.item = self.parser.match_item("branch", self.character.inventory)

    def check_preconditions(self) -> bool:

        if hasattr(self.character, 'inventory'):
            inventory_keys = list(self.character.inventory.keys())

        if not self.target:
            self.parser.fail("There's no guard here to attack.")
            return False

        if not self.target.get_property("is_conscious"):
            self.parser.fail("The guard is already unconscious.")
            return False

        return True

    def apply_effects(self):

        self.target.set_property("is_conscious", False)
        self.target.set_property("has_key", False)

        try:
            if "east" in courtyard.blocks:
                del courtyard.blocks["east"]
                print("East block removed!")
        except:
            pass

        try:
            courtyard.remove_block("east")

        except:
            pass

        if "key" in self.target.inventory:
            item = self.target.inventory.pop("key")
            courtyard.add_item(item)

        self.parser.ok(
            "You strike the guard with the branch! The guard collapses, unconscious. The key falls to the ground."
        )


class PickRose(actions.Action):
    ACTION_NAME = "pick"
    ACTION_DESCRIPTION = "Pick a rose from the rosebush"

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.rosebush = self.parser.match_item(
            "rosebush", self.parser.get_items_in_scope(self.character))

    def check_preconditions(self) -> bool:
        if not self.rosebush or not self.rosebush.get_property("has_rose"):
            self.parser.fail("There is no rose to pick here!")
            return False
        return True

    def apply_effects(self):
        self.rosebush.set_property("has_rose", False)
        self.character.add_to_inventory(rose)
        self.parser.ok("You pick a beautiful red rose from the rosebush.")


class LightLamp(actions.Action):
    ACTION_NAME = "light"
    ACTION_DESCRIPTION = "Light the lamp to illuminate the area"

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.item = self.parser.match_item("lamp", self.character.inventory)

    def check_preconditions(self) -> bool:
        if not self.item or not self.item.get_property("is_light_source"):
            self.parser.fail("You need a lamp to light!")
            return False
        if self.item.get_property("is_lit"):
            self.parser.fail("The lamp is already lit!")
            return False
        return True

    def apply_effects(self):
        self.item.set_property("is_lit", True)

        try:
            if "down" in dungeon_stairs.blocks:
                del dungeon_stairs.blocks["down"]
                print("Darkness block removed!")
        except:
            pass

        try:
            dungeon_stairs.remove_block("down")

        except:
            pass

        self.parser.ok(
            "You light the lamp. The dungeon stairs are now safe to descend!")


class UnlockDoor(actions.Action):
    ACTION_NAME = "unlock"
    ACTION_DESCRIPTION = "Unlock the tower door with a key"

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.item = self.parser.match_item("key", self.character.inventory)

    def check_preconditions(self) -> bool:
        if not self.item or not self.item.get_property("is_key"):
            self.parser.fail("You need a key to unlock the door.")
            return False
        if not tower_stairs.is_blocked("up"):
            self.parser.fail("The tower door is already unlocked.")
            return False
        return True

    def apply_effects(self):
        tower_stairs.remove_block(door_block)
        self.parser.ok("You unlock the tower door. The way up is now clear.")


class ReadRunes(actions.Action):
    ACTION_NAME = "read"
    ACTION_DESCRIPTION = "Read the runes to banish the ghost in the dungeon"

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)

    def check_preconditions(self) -> bool:
        ghost_found = any(char.name.lower(
        ) == "ghost" for char in self.character.location.characters.values())
        if not ghost_found:
            self.parser.fail("There are no runes to read here.")
            return False
        return True

    def apply_effects(self):
        dungeon.remove_character(ghost)
        self.parser.ok(
            "You read the ancient runes aloud. The ghost wails and vanishes!")


class ProposeMarriage(actions.Action):
    ACTION_NAME = "propose"
    ACTION_DESCRIPTION = "Propose marriage to the princess"

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.princess = None
        self.rose = None

        for char_name, char in self.character.location.characters.items():
            if "princess" in char_name.lower():
                self.princess = char
                break

        if hasattr(self.character, 'inventory') and self.character.inventory:

            if "rose" in self.character.inventory:
                self.rose = self.character.inventory["rose"]

            if not self.rose:
                for item_name, item in self.character.inventory.items():
                    if hasattr(item, 'get_property') and item.get_property("is_gift"):
                        if "rose" in item_name.lower() or "rose" in str(item).lower():
                            self.rose = item

                            break

            if not self.rose:
                for item_name, item in self.character.inventory.items():
                    if "rose" in item_name.lower():
                        self.rose = item

                        break

    def check_preconditions(self) -> bool:

        if not self.princess:
            self.parser.fail("There is no princess here to propose to!")
            return False

        is_royal = self.character.get_property("is_royal")
        if not is_royal:
            self.parser.fail("You are not royal! She won't accept.")
            return False

        if not self.rose:
            self.parser.fail("You need a rose to propose!")
            return False

        return True

    def apply_effects(self):
        self.parser.ok(
            "The princess accepts your proposal! You are now married!")
        self.character.set_property("is_married", True)


class WearCrown(actions.Action):
    ACTION_NAME = "wear"
    ACTION_DESCRIPTION = "Wear the crown to become royal"

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.item = self.parser.match_item("crown", self.character.inventory)

    def check_preconditions(self) -> bool:
        if not self.item or not self.item.get_property("is_crown"):
            self.parser.fail("You don't have a crown to wear.")
            return False

        is_wearing_crown = self.character.get_property("is_wearing_crown")
        if is_wearing_crown:
            self.parser.fail("You are already wearing the crown.")
            return False
        return True

    def apply_effects(self):
        self.character.set_property("is_wearing_crown", True)
        self.character.set_property("is_royal", True)

        self.parser.ok("You put on the crown. You feel royal!")


class SitThrone(actions.Action):
    ACTION_NAME = "sit"
    ACTION_DESCRIPTION = "Sit on the throne and claim victory"

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.throne = self.parser.match_item(
            "throne", self.parser.get_items_in_scope(self.character))

    def check_preconditions(self) -> bool:
        if not self.throne:
            self.parser.fail("There is no throne here to sit on!")
            return False

        is_royal = self.character.get_property("is_royal")
        if not is_royal:
            self.parser.fail("You must be royal to sit on the throne!")
            return False

        is_wearing_crown = self.character.get_property("is_wearing_crown")
        if not is_wearing_crown:
            self.parser.fail("You must be wearing the crown!")
            return False
        return True

    def apply_effects(self):
        self.parser.ok(
            "You sit on the throne. You are now the ruler of the castle!")
        self.character.set_property("is_king", True)


class ActionCastleGame(games.Game):
    def __init__(self, start_at: things.Location, player: things.Character, characters=None, custom_actions=None):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)
        self.player.set_property("is_hungry", True)

    def is_won(self) -> bool:
        is_king = self.player.get_property("is_king")
        if is_king:
            self.parser.ok("You have claimed the throne! You win the game!")
            return True
        return False

## Commands

In text adventure games the user types in simple natural language commands to interact with the game environment. That text is parsed for meaning and actions may or may not take place as a result.

One of the challenges for the parser is mapping the language to corresponding components of the game. Classic text adventure games use simple two word commands, eg. _go east_ or _get pole_.

In the next section, you will see five commands that will guide you through our game.

## Let's Play!

*Behold, as a magestic prompt appears!*

You can play through the whole game with the following commands:

1. take pole
2. go out
3. south
4. catch fish with pole
5. eat fish

In [11]:
game = ActionCastleGame(
    cottage,
    player,
    characters=[guard, princess, ghost],
    custom_actions=[HitGuard, PickRose, LightLamp, UnlockDoor,
                    ReadRunes, ProposeMarriage, WearCrown, SitThrone]
)


print("Welcome to Action Castle!")
print("Your goal is to become the king by sitting on the throne.")
print("Type 'help' for available commands or 'quit' to exit.\n")

game.game_loop()

Welcome to Action Castle!
Your goal is to become the king by sitting on the throne.
Type 'help' for available commands or 'quit' to exit.

You are standing in a small cottage.
Exits:
Out to Garden Path

You see:
 * a fishing pole
 * a poisonous potion
        drink potion
 * a glowing lamp


The player got the lamp.
You are standing on a lush garden path. There is a cottage here.
Exits:
In to Cottage
South to Fishing Pond
North to Courtyard

You see:
 * a sturdy tree branch
 * a rosebush
        pick rose


No action found for out
I'm not sure what you want to do.
The player got the branch.
The player picked the lone rose from the rosebush
The rosebush is bare.
You are standing in the castle courtyard. A guard blocks the way east.
Exits:
South to Garden Path
West to Drawbridge
Up to Tower Stairs
Down to Dungeon Stairs
East to Great Feasting Hall


Characters:
 * A stern guard in chainmail blocks the way east. He wears no helmet. A key
hangs from his belt.

East block removed!
You strik

### Visualize Game

One way of conceptualizing our games is as a directed graph, where nodes are locations connected via directed arcs.  We can vizualize our game as a directed graph.

In [ ]:
from text_adventure_games.viz import Visualizer
viz = Visualizer(game)
graph = viz.visualize()
graph

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b16fb3c8-85ab-4463-b29e-8658c8738191' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>